## 测试 mci_gru 框架

In [1]:
from main import *

In [14]:
batch_size = 100
D = 10 #原始的单日的特征维度数
T = 30 #单个样本用于预测所使用的数据的天数
E = 10 #映射后的特征维度数
U = 10 #LSTM网络中的隐藏维度数
Q = 10
num_lstmlayer = 1#LSTM的层数
beta = 1 #对抗学习损失函数部分的权重

#模拟一个batch的数据输入
origin_input = torch.randn(batch_size,T,D)
y_label = torch.ones(batch_size)

In [19]:
LSTM_layer = nn.LSTM(E,U,num_lstmlayer,batch_first=True)

In [23]:
output, (hn, cn) = LSTM_layer(origin_input)

In [24]:
hn.shape

torch.Size([1, 100, 10])

In [25]:
output.shape

torch.Size([100, 30, 10])

In [46]:
a_s = torch.randn(batch_size,U)

In [48]:
torch.cat([a_s,output[:,-1,:]],dim = 1).shape

torch.Size([100, 20])

In [34]:
output.transpose(1,2).shape

torch.Size([100, 10, 30])

In [37]:
weighted_vector.shape

torch.Size([100, 30, 1])

In [ ]:
class ALSTM():
    def __init__(self,
                input_size,
                embedding_dim, 
                hidden_size, 
                Q, 
                num_lstmlayer = 1,
                beta = 1):
        super(ALSTM, self).__init__()
        self.input_size = input_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.Q = Q
        self.num_lstmlayer = num_lstmlayer
        self.beta = beta
        self.MappingLayer = nn.Linear(input_size,embedding_dim)
        self.tanh = nn.Tanh()
        self.LSTMLayer = nn.LSTM(embedding_dim,hidden_size,num_lstmlayer,batch_first=True)
        self.fc1 = nn.Linear(hidden_size,Q)
        self.fc2 = nn.Linear(Q, 1, bias=False)
        self.final_map = nn.Linear(hidden_size*2,1)

    def AttentionLayer(self, h_s):
        x = self.tanh(self.fc1(h_s)) 
        x = self.fc2(x)
        weighted_vector = torch.exp(x)/torch.sum(torch.exp(x),dim = 1).unsqueeze(1)  #weight of each time stamp
        a_s = torch.bmm(output.transpose(1,2),weighted_vector).squeeze(2)
        return a_s

    def forward(self, input):
        input = self.tanh(self.MappingLayer(input))  # 先映射到隐藏层空间
        output, (hn, cn) = self.LSTMLayer(input)
        a_s = self.AttentionLayer(output)
        e_s = torch.cat([a_s,output[:,-1,:]],dim = 1)
        y = self.tanh(self.final_map(e_s)) #输出为(-1,1)之间的值
        return y 

def get_adv(origin_input,y_label,final_map,model,epsilon,criterion):
    '''
    origin_input: batch_size * T * feature_dim
    y_label: batch_size
    final_map: final mapping layer
    model: Attentive lSTM
    epsilon: learning rate to control the adv examples
    criterion: loss function
    '''
    e_s = model(origin_input)
    e_s.retain_grad()
    y_s = torch.sign(final_map(e_s)).squeeze(1)
    loss_1 = criterion(y_s,y_label)
    g_s = torch.autograd.grad(outputs = loss_1,inputs=e_s,grad_outputs=None)[0]
    g_snorm = torch.sqrt(torch.norm(g_s,p = 2))
    if g_snorm == 0:
        return 0
    else:
        r_adv = epsilon(g_s/g_snorm)
        return r_adv.detach()
    
class HingeLoss(nn.Module):
    def __init__(self):
        super(HingeLoss,self).__init__()
    def forward(self,y_label,y_predict):
        #y_label,y_predict都是长度为batch_size的一维tensor
        if y_label.shape == y_predict.shape:
            # 这里可能需要加上一个时间权重
            loss = torch.sum(torch.max(torch.zeros_like(y_label),torch.ones_like(y_label)-y_label * y_predict))
            return loss
        else:
            raise Exception("The size of label and predicted value is not equal !")

In [67]:
from main import *

In [52]:
localpath = '.'
AllData = []
for data_name in ['Train_data', 'Train_label', 'Valid_data', 'Valid_label', 'Test_data', 'Test_label']:
    with open(f'{localpath}/data/{data_name}.pickle', 'rb') as file:
        loaded_data = pickle.load(file)
    AllData.append(loaded_data)
Train_data, Train_label, Valid_data, Valid_label, Test_data, Test_label = tuple(AllData)


In [53]:
Train_data.shape

(13930, 30, 7)

In [64]:
TrainBatchSampler = RandomBatchSampler(Train_label)
ValidBatchSampler = RandomBatchSampler(Valid_label)
TrainDataloader = DataLoader(MyDataset(Train_data, Train_label['htc_ret'].values), batch_sampler=TrainBatchSampler, pin_memory=True)
ValidDataloader = DataLoader(MyDataset(Valid_data, Valid_label['htc_ret'].values), batch_sampler=ValidBatchSampler, pin_memory=True)
TestDataloader = DataLoader(MyDataset(Test_data, Test_label['htc_ret'].values), pin_memory=True)


In [59]:
Train_label['time'].dt.date.drop_duplicates()

0       2022-02-18
12      2022-02-21
24      2022-02-22
36      2022-02-23
48      2022-02-24
           ...    
5595    2023-12-29
5607    2022-02-14
5623    2022-02-15
5641    2022-02-16
5659    2022-02-17
Name: time, Length: 460, dtype: object

In [61]:
Train_label[Train_label['Fund_code']== 'F588080.SH']

,time,Fund_code,htc_ret,date
0,2022-02-18 10:35:00,F588080.SH,1.0,2022-02-18
1,2022-02-18 10:45:00,F588080.SH,1.0,2022-02-18
2,2022-02-18 10:55:00,F588080.SH,0.0,2022-02-18
3,2022-02-18 11:05:00,F588080.SH,1.0,2022-02-18
4,2022-02-18 11:15:00,F588080.SH,1.0,2022-02-18
...,...,...,...,...
5602,2023-12-29 13:10:00,F588080.SH,1.0,2023-12-29
5603,2023-12-29 13:20:00,F588080.SH,1.0,2023-12-29
5604,2023-12-29 13:30:00,F588080.SH,1.0,2023-12-29
5605,2023-12-29 13:40:00,F588080.SH,1.0,2023-12-29


In [65]:
summ = 0
for inputs, labels in ValidDataloader:
    summ+=1
print(summ)

56


In [7]:
mapping_layer = nn.Linear(D,E,bias = True)
origin_input = origin_input.flatten(start_dim = 0,end_dim = 1)

In [72]:
class ALSTM(nn.Module):
    def __init__(self,
                input_size,
                embedding_dim, 
                hidden_size, 
                Q, 
                num_layers = 1,
                beta = 1):
        super(ALSTM, self).__init__()
        self.input_size = input_size
        self.embedding_dim = embedding_dim
        self.hidden_size = hidden_size
        self.Q = Q
        self.num_layers = num_layers
        self.beta = beta
        self.MappingLayer = nn.Linear(input_size,embedding_dim)
        self.tanh = nn.Tanh()
        self.LSTMLayer = nn.LSTM(embedding_dim,hidden_size,num_layers,batch_first=True)
        self.fc1 = nn.Linear(hidden_size,Q)
        self.fc2 = nn.Linear(Q, 1, bias=False)
        self.final_map = nn.Linear(hidden_size*2,1)

    def AttentionLayer(self, h_s):
        x = self.tanh(self.fc1(h_s)) 
        x = self.fc2(x)
        weighted_vector = torch.exp(x)/torch.sum(torch.exp(x),dim = 1).unsqueeze(1)  #weight of each time stamp
        a_s = torch.bmm(h_s.transpose(1,2),weighted_vector).squeeze(2)
        return a_s

    def forward(self, input):
        input = self.tanh(self.MappingLayer(input))  # 先映射到隐藏层空间
        output, (hn, cn) = self.LSTMLayer(input)
        a_s = self.AttentionLayer(output)
        e_s = torch.cat([a_s,output[:,-1,:]],dim = 1)
        y = self.tanh(self.final_map(e_s)) #输出为(-1,1)之间的值
        return y 

class HingeLoss(nn.Module):
    def __init__(self):
        super(HingeLoss,self).__init__()
    def forward(self,y_label,y_predict):
        #y_label,y_predict都是长度为batch_size的一维tensor
        if y_label.shape == y_predict.shape:
            # 这里可能需要加上一个时间权重
            loss = torch.sum(torch.max(torch.zeros_like(y_label),torch.ones_like(y_label)-y_label * y_predict))
            return loss
        else:
            print(y_label.shape)
            print(y_predict.shape)
            raise Exception("The size of label and predicted value is not equal !")
    
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with open(f'{localpath}/test1.yaml', 'r') as file:
    args = DictToObj(yaml.safe_load(file))
model = ALSTM(input_size=len(args.FACTOR_LIST),
            embedding_dim = args.EMBEDDING_DIM, 
            hidden_size=args.HIDDEN_SIZE, 
            Q = args.Q, 
            num_layers=args.NUM_LAYERS)
model.to(device)
criterion = HingeLoss()

In [74]:
def validate(factor_model, dataloader, criterion):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    factor_model.to(device)
    factor_model.eval()
    total_loss = 0
    with tqdm(total=len(dataloader), desc="Validation") as pbar:
        for inputs, labels  in dataloader:
            inputs = inputs.float().to(device)
            labels = labels.float().to(device)
            y_pred = factor_model(inputs)
            loss = criterion(labels.unsqueeze(1), y_pred)
            #total_loss += loss.item() 
            total_loss += 1
            pbar.update(1)
    avg_loss = total_loss / len(dataloader)
    return total_loss

In [75]:
validate(model, ValidDataloader, criterion)

Validation:   0%|          | 0/56 [00:00<?, ?it/s]

Validation: 100%|██████████| 56/56 [00:00<00:00, 108.44it/s]


56

In [9]:
origin_input.shape

torch.Size([3000, 10])

In [4]:
mapping_layer = nn.Linear(D,E,bias = True)
origin_input = origin_input.flatten(start_dim = 0,end_dim = 1)
tanh = nn.Tanh()
mapping_feature = tanh(mapping_layer(origin_input))
mapping_feature = mapping_feature.reshape(batch_size,T,-1)
mapping_feature = mapping_feature.transpose(0,1)

In [5]:
mapping_feature.shape

torch.Size([30, 100, 10])

In [3]:
y_label

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [ ]:
def make_dataset(F588080_temp, F159915_temp, seq_len, seq_gap, factor_list, label_name):
    data = []
    label = []
    for Fund_temp in [F588080_temp, F159915_temp]:
        Fund_temp = Fund_temp[Fund_temp.index.strftime('%H:%M')<='14:00']
        for i in tqdm(range(seq_len, len(Fund_temp), seq_gap)):
            temp_data = Fund_temp.iloc[i-seq_len:i][factor_list].values
            temp_label = Fund_temp.iloc[i-1][label_name]
            if np.isnan(temp_data).sum()+np.isnan(temp_label) == 0:
                data.append(temp_data)
                label.append(temp_label)
    return np.array(data), np.array(label)

In [9]:
F588080_Train

,windcode,open,high,low,close,volume,amount,change,pctchange,htc_ret
2022-02-14 09:30:00,588080.SH,1.200,1.200,1.190,1.190,4578800.0,5473856.0,-0.014,-0.011628,0.013434
2022-02-14 09:31:00,588080.SH,1.191,1.191,1.185,1.186,3978803.0,4719635.0,-0.004,-0.003361,0.017707
2022-02-14 09:32:00,588080.SH,1.186,1.194,1.185,1.193,2997000.0,3566074.0,0.007,0.005902,0.011735
2022-02-14 09:33:00,588080.SH,1.193,1.198,1.193,1.198,2019600.0,2412967.0,0.005,0.004191,0.006672
2022-02-14 09:34:00,588080.SH,1.199,1.202,1.198,1.202,3673700.0,4409735.0,0.004,0.003339,0.004160
...,...,...,...,...,...,...,...,...,...,...
2023-12-29 14:55:00,588080.SH,0.873,0.873,0.872,0.872,1099900.0,959909.0,0.000,0.000000,-0.001145
2023-12-29 14:56:00,588080.SH,0.873,0.873,0.872,0.872,1330800.0,1161037.0,0.000,0.000000,0.000000
2023-12-29 14:57:00,588080.SH,0.872,0.873,0.872,0.872,1802407.0,1572105.0,0.000,0.000000,-0.001145
2023-12-29 14:58:00,588080.SH,0.873,0.873,0.872,0.873,2035000.0,1776124.0,0.001,0.001147,0.000000


In [8]:
with open('test1.yaml', 'r') as file:
    args = DictToObj(yaml.safe_load(file))
ETF_minquotes = pd.read_parquet('ETF_minquotes.parquet')
F588080 = ETF_minquotes.loc[ETF_minquotes['windcode'] == '588080.SH']
F159915 = ETF_minquotes.loc[ETF_minquotes['windcode'] == '159915.SZ']
F588080.loc[:,'htc_ret'] = F588080.groupby(F588080.index.date).apply(lambda x:x['close'].iloc[-1]/x['open'].shift(-1)-1).values
F159915.loc[:,'htc_ret'] = F159915.groupby(F159915.index.date).apply(lambda x:x['close'].iloc[-1]/x['open'].shift(-1)-1).values

F588080_Train = F588080.loc[(F588080.index.date >= args.TRAIN.START_DATE) & (F588080.index.date <= args.TRAIN.END_DATE)]
F159915_Train = F159915.loc[(F159915.index.date >= args.TRAIN.START_DATE) & (F159915.index.date <= args.TRAIN.END_DATE)]
F588080_Valid = F588080.loc[(F588080.index.date >= args.VALID.START_DATE) & (F588080.index.date <= args.VALID.END_DATE)]
F159915_Valid = F159915.loc[(F159915.index.date >= args.VALID.START_DATE) & (F159915.index.date <= args.VALID.END_DATE)]
F588080_Test = F588080.loc[(F588080.index.date >= args.TEST.START_DATE) & (F588080.index.date <= args.TEST.END_DATE)]
F159915_Test = F159915.loc[(F159915.index.date >= args.TEST.START_DATE) & (F159915.index.date <= args.TEST.END_DATE)]

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_27052\2034361835.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  F588080.loc[:,'htc_ret'] = F588080.groupby(F588080.index.date).apply(lambda x:x['close'].iloc[-1]/x['open'].shift(-1)-1).values
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_27052\2034361835.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  F159915.loc[:,'htc_ret'] = F159915.groupby(F159915.index.date).apply(lambda x:x['close'].iloc[-1]/x['open'].shift(-1)-1).values


In [ ]:
class AGRU(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(AGRU, self).__init__()
        self.hidden_size = hidden_size

        # 输入到隐藏层的权重
        self.weight_ih = nn.Parameter(torch.randn(3 * hidden_size, input_size))
        # 隐藏层到隐藏层的权重
        self.weight_hh = nn.Parameter(torch.randn(3 * hidden_size, hidden_size))
        # 偏置项
        self.bias_ih = nn.Parameter(torch.randn(3 * hidden_size))
        self.bias_hh = nn.Parameter(torch.randn(3 * hidden_size))

    def forward(self, input):
        """
        input: (seq_len, batch_size, input_size)
        h_prev: (batch_size, hidden_size)
        """
        seq_len, batch_size, input_size = input.shape
        hidden_size = self.hidden_size
        h_prev = torch.zeros(batch_size, hidden_size)
        # 初始化输出和隐藏状态
        outputs = []
        h_next = h_prev

        for t in range(seq_len):
            x_t = input[t]  # (batch_size, input_size)

            # 计算输入和隐藏状态的线性变换
            gates_ih = torch.mm(x_t, self.weight_ih.t()) + self.bias_ih  # (batch_size, 3 * hidden_size)
            gates_hh = torch.mm(h_next, self.weight_hh.t()) + self.bias_hh  # (batch_size, 3 * hidden_size)

            # 拆分线性变换的结果
            r_gate, z_gate, n_gate = gates_ih.chunk(3, 1)  # 每部分形状: (batch_size, hidden_size)
            r_gate_h, z_gate_h, n_gate_h = gates_hh.chunk(3, 1)

            # 计算重置门、更新门和候选隐藏状态
            r_gate = torch.sigmoid(r_gate + r_gate_h)
            z_gate = torch.sigmoid(z_gate + z_gate_h)
            n_gate = torch.tanh(n_gate + r_gate * n_gate_h)

            # 更新隐藏状态
            h_next = (1 - z_gate) * n_gate + z_gate * h_next

            # 保存当前时间步的隐藏状态
            outputs.append(h_next.unsqueeze(0))  # (1, batch_size, hidden_size)

        # 将所有时间步的隐藏状态拼接起来
        outputs = torch.cat(outputs, dim=0)  # (seq_len, batch_size, hidden_size)

        # 返回所有时间步的输出和最后一步的隐藏状态
        return outputs, h_next

In [1]:
import pickle
localpath = 'D:/PythonProject1/ETF_T0'
data_name = 'Train_label'

In [6]:
F588080[:240]

,windcode,open,high,low,close,volume,amount,change,pctchange
2022-02-14 09:30:00,588080.SH,1.200,1.200,1.190,1.190,4578800.0,5473856.0,-0.014,-0.011628
2022-02-14 09:31:00,588080.SH,1.191,1.191,1.185,1.186,3978803.0,4719635.0,-0.004,-0.003361
2022-02-14 09:32:00,588080.SH,1.186,1.194,1.185,1.193,2997000.0,3566074.0,0.007,0.005902
2022-02-14 09:33:00,588080.SH,1.193,1.198,1.193,1.198,2019600.0,2412967.0,0.005,0.004191
2022-02-14 09:34:00,588080.SH,1.199,1.202,1.198,1.202,3673700.0,4409735.0,0.004,0.003339
...,...,...,...,...,...,...,...,...,...
2022-02-14 14:55:00,588080.SH,1.203,1.205,1.202,1.205,451000.0,542913.0,0.002,0.001662
2022-02-14 14:56:00,588080.SH,1.205,1.206,1.204,1.206,718100.0,865193.0,0.001,0.000830
2022-02-14 14:57:00,588080.SH,1.206,1.206,1.205,1.206,483700.0,583308.0,0.000,0.000000
2022-02-14 14:58:00,588080.SH,1.206,1.207,1.206,1.207,642800.0,775414.0,0.001,0.000829


In [2]:
with open(f'./data/{data_name}.pickle', 'rb') as file:
    loaded_data = pickle.load(file)

EOFError: Ran out of input

In [9]:
file

<_io.BufferedReader name='D:/PythonProject1/ETF_T0/data/Train_label.pickle'>